[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/gray_compression/rate-control_deadzone.ipynb)

# Rate-control deadzone

In [ ]:
!ln -sf ~/MRVC/src/logging_config.py .
!ln -sf ~/repos/quantization/deadzone_quantizer.py .
!ln -sf ~/repos/quantization/distortion.py .
!ln -sf ~/MRVC/src/image_1.py .
!ln -sf ~/repos/quantization/information.py .

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
#plt.rcParams['text.usetex'] = True
#plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
import pylab
import math
import numpy as np
from scipy import signal
import cv2
import os
import deadzone_quantizer as deadzone
import distortion
import image_1 as gray_image
import colored
import information

## Configuration

In [ ]:
# Prefix of the RGB image to be quantized.

home = os.environ["HOME"]
fn = home + "/MRVC/images/lena_bw/"

quantizer = deadzone

# Maybe this does not make sense in digital quantization (generate loops in the RD domain
# because the output number of quantization indexes is not a power of 2.)
Q_steps = range(128, 0, -1)

#Q_steps = [2**i for i in range(7, -1, -1)] # Quantization steps (simulating bit-plane encoding)

#quantizer = companded
#Q_steps = [2**i for i in range(16, -1, -1)] # Quantization steps (simulating bit-plane encoding)

print(Q_steps)

#RGB_image.write = RGB_image.debug_write # faster
#RGB_image.write = RGB_image.write # higher compression

gray_image.write = gray_image.debug_write # faster
#gray_image.write = gray_image.write # higher compression

Notice that non embbeded quatization (using steps thar are different from a power of 2) steps (can produce loops in the RD curves due to the non-linearity of the integer division performed in the quantization).

## Read the image and show it

In [ ]:
img = gray_image.read(fn, 0).astype(np.int16) # 8 bits/components is not sufficient to shift the components to [-128, 127]
print(img.dtype)
print(img.max(), img.min())
gray_image.show(img, fn + "000.png")

## RD curve

In [ ]:
def RD_curve(_img, Q_steps, quantizer):
    img = _img.copy()
    img -= 128
    points = []
    for Q_step in Q_steps:
        y, k = quantizer.quan_dequan(img, Q_step)
        k = (k + 128).astype(np.uint8) # Only positive components can be written in an PNG file
        print("Quantization indexes: ", np.unique(k))
        rate = gray_image.write(k, "/tmp/" + str(Q_step) + '_', 0)*8/k.size
        _distortion = distortion.RMSE(img, y)
        points.append((rate, _distortion))
        print(f"q_step={Q_step:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

RD_points = RD_curve(img, Q_steps, quantizer)

In [ ]:
def RD_curve(_img, Q_steps, quantizer):
    img = _img.copy()
    img -= 128
    points = []
    for Q_step in Q_steps:
        y, k = quantizer.quan_dequan(img, Q_step)
        k = (k + 128).astype(np.uint8) # Only positive components can be written in an PNG file
        print("Quantization indexes: ", np.unique(k))
        rate = information.entropy(k.flatten())
        _distortion = distortion.RMSE(img, y)
        points.append((rate, _distortion))
        print(f"q_step={Q_step:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

entropy_RD_points = RD_curve(img, Q_steps, quantizer)

In [ ]:
deadzone_RD = []
with open(f"dead-zone_RD_points.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        deadzone_RD.append((float(rate), float(_distortion)))

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*deadzone_RD), c='g', marker='o', label=f"{quantizer.name} PNG (bit-planes)", linestyle="dashed")
pylab.scatter(*zip(*RD_points), label=f"{quantizer.name} PNG", s=1, marker='.')
pylab.scatter(*zip(*entropy_RD_points), label=f"{quantizer.name} entropy", s=1, marker='.')
pylab.title(f"Rate/Distortion Performance ({quantizer.name})")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()